In [ ]:
from torchvision.transforms.transforms import RandomHorizontalFlip
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm
import torch.nn.functional as F

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [ ]:
transformer = transforms.Compose(
                                 [transforms.Resize(256),
                                  transforms.RandomCrop(224),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.ColorJitter(brightness=0.2,hue=0.2),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                 ])

train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transformer)
val_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transformer)
x_train,y_train=train_data.data,train_data.targets

In [ ]:
plt.imshow(x_train[0])
x_train[0].shape

In [ ]:
BATCH_SIZE = 256
VAL_BATCH_SIZE = 500
train_loader = torch.utils.data.DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=False,num_workers=1)
val_loader = torch.utils.data.DataLoader(val_data,batch_size=VAL_BATCH_SIZE,shuffle=True,num_workers=1)

In [ ]:
class My_ResNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
        )

        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
        )
        
        self.conv_layer3 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
        )
        
        self.conv_layer4 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
        )
        
        self.conv_layer5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
        )
        
        self.conv_layer3_start = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
        )
        
        self.conv_layer4_start = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
        )
        
        self.conv_layer5_start = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
        )
        
        self.avgpool = nn.AvgPool2d(kernel_size=3, stride=2, padding=1)
        self.fc = nn.Linear(512*512*13, self.num_classes)
        self.dimension_to128 = nn.Conv2d(64,128,kernel_size=1, stride=2)
        self.dimension_to256 = nn.Conv2d(128,256,kernel_size=1, stride=2)
        self.dimension_to512 = nn.Conv2d(256,512,kernel_size=1, stride=2)
    def forward(self, x):
        # First Conv & Pooling Layer
        x = self.conv_layer1(x)
        
        # Second Layer
        for i in range(3):
            res_x = x
            x = self.conv_layer2(x)
            x = x + res_x
            x = F.relu(x)
        print(x.shape)
        # Third Layer (Only Conv and ReLU)
        res_x = self.dimension_to128(x)
        x = self.conv_layer3_start(x)
        x = x + res_x
        x = F.relu(x)
        for i in range(3):
            res_x = x
            x = self.conv_layer3(x)
            x = x + self.dimension_to128(res_x)
            x = F.relu(x)
        
        # Fourth Layer (Only Conv and ReLU)
        res_x = self.dimension_to256(x)
        x = self.conv_layer4_start(x)
        x = x + res_x
        x = F.relu(x)
        for i in range(5):
            res_x = x
            x = self.conv_layer4(x)
            x = x + res_x
            x = F.relu(x)
        
        # Fifth Layer (Only Conv and ReLU)
        res_x = self.dimension_to512(x)
        x = self.conv_layer5_start(x)
        x = x + res_x
        x = F.relu(x)
        for i in range(2):
            res_x = x
            x = self.conv_layer5(x)
            x = x + res_x
            x = F.relu(x)
        
        # Output: 256*6*6

        x = self.avgpool(x)
        x = x.view(-1, 512*512*13)
        return self.fc(x)

        

In [ ]:
device='cuda'
import matplotlib.pyplot as plt
import numpy as np

for data,label in train_loader:
    data=data.to(device)
    print(data.shape)
#     print(data[0].shape,data[0])
#     img = data[0].numpy()
#     plt.imshow(np.transpose(img,(1,2,0))) # 3, 224, 224 -> 224, 224, 3으로 size 변경
# #     axs.axis('off')

    break

In [ ]:
import torch.optim as optim
from tqdm.auto import tqdm

learning_rate = 1e-4
momentum = 0.9
weight_decay = 0.0005
model = My_ResNet(10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

model.train()
epochs = 5
total_batch = len(train_loader)
for epoch in range(epochs):
  avg_cost = 0
  for img, label in train_loader:
    img = img.to(device)
    label = label.to(device)

    output = model(img)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    avg_loss += loss/total_batch
  
  print('[Epoch: {:>4}] loss = {:>.9}'.format(epoch + 1, avg_loss))